In [ ]:
from google.colab import drive
drive.mount("/content/drive")

# 1.0 Install Mask RCNN library

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/HD_object_recognition/Mask_RCNN
!python setup.py install

/content/drive/My Drive/Colab Notebooks/ObjectRecognition/Mask_RCNN
running install
running bdist_egg
running egg_info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/config.py -> build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/__init__.py -> build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/model.py -> build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/utils.py -> build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/parallel_model.py -> build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/visualize.py -> build/bdist.linux-x86_64/egg/mrcnn
byte

# 1.1 Install compatible version of tensorflow

In [ ]:
#!pip install tensorflow==1.14
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

tensorflow.compat.v1.logging.set_verbosity(tensorflow.compat.v1.logging.ERROR)

TensorFlow 1.x selected.
1.15.2


# 1.2 Load data into an MRCNN Dataset() class

In [ ]:
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN


class SkinDataset(Dataset):

	def load_dataset(self, dataset_dir, is_train=True):
		# define features
		self.add_class("dataset", 1, "follicular_bump")
		self.add_class("dataset", 2, "pre_erupted")
		# define data locations
		images_dir = dataset_dir + '/training images/'
		annotations_dir = dataset_dir + '/training annots/'
		# find all images
		for filename in listdir(images_dir):
			image_id = filename[0:-4]
			last_dig = filename[4:-4]
			if image_id in ['']:
				continue
			# separate 20% of data for validation set
			if is_train and int(last_dig) in [1, 6]:
				continue
			if not is_train and int(last_dig) not in [1, 6]:
				continue
			img_path = images_dir + filename
			ann_path = annotations_dir + image_id + '.xml'
			# add to dataset
			self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])

	def extract_boxes(self, filename):
		# load and parse the file
		tree = ElementTree.parse(filename)
		root = tree.getroot()
		# extract each bounding box
		boxes = list()
		for box in root.findall('.//object'):
			name = box.find('name').text
			## if name == 'follicular_bump':
			xmin = int(box.find('./bndbox/xmin').text)
			ymin = int(box.find('./bndbox/ymin').text)
			xmax = int(box.find('./bndbox/xmax').text)
			ymax = int(box.find('./bndbox/ymax').text)
			coors = [xmin, ymin, xmax, ymax, name]
			boxes.append(coors)
		# extract image dimensions
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	def load_mask(self, image_id):
		# get details of image
		info = self.image_info[image_id]
		# define box file location
		path = info['annotation']
		# load XML
		boxes, w, h = self.extract_boxes(path)
		# create one array for all masks, each on a different channel
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			if (box[4] == 'follicular_bump'):
				masks[row_s:row_e, col_s:col_e, i] = 1
				class_ids.append(self.class_names.index('follicular_bump'))
			else:
				masks[row_s:row_e, col_s:col_e, i] = 2
				class_ids.append(self.class_names.index('pre_erupted'))
		return masks, asarray(class_ids, dtype='int32')

	# load an image reference
	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']


# prepare training set
train_set = SkinDataset()
train_set.load_dataset('skin data', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

# prepare test set
test_set = SkinDataset()
test_set.load_dataset('skin data', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))


# 1.3 Display an image with a bounding box

In [ ]:
from matplotlib import pyplot

IMAGE_ID = 1
OBJECT_ID = 0

# load image and mask
image = train_set.load_image(IMAGE_ID)
mask, class_ids = train_set.load_mask(IMAGE_ID)
print('image shape: ', image.shape)
print('mask shape:  ', mask.shape)

# plot image and mask
pyplot.figure(figsize=(15,10))
pyplot.imshow(image)
pyplot.imshow(mask[:, :, OBJECT_ID], cmap='gray', alpha=0.33)
pyplot.show()

# 1.4 Display an image with masks

In [ ]:
from mrcnn.utils import Dataset
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes

IMAGE_ID = 1

# load the image, masks and class ids
image = train_set.load_image(IMAGE_ID)
mask, class_ids = train_set.load_mask(IMAGE_ID)

# extract bounding boxes from the masks
bbox = extract_bboxes(mask)

# display image with masks and bounding boxes
display_instances(image, bbox, mask, class_ids, train_set.class_names)

ModuleNotFoundError: ignored

# 1.5  Retrain MRCNN fitted on MSCOCO dataset with skin dataset

In [ ]:
class SkinConfig(Config):
	NAME = "skin11cfg"
	NUM_CLASSES = 3         # include background (class 0)
	EPOCHS = 10
	STEPS_PER_EPOCH = 1159
	VALIDATION_STEPS = 290
	IMAGE_RESIZE_MODE = 'square'
	IMAGE_MIN_DIM = 1472
	IMAGE_MAX_DIM = 1728    # should be divisible by 64
	IMAGES_PER_GPU = 1
	LEARNING_RATE = 0.001   # 0.001

# prepare config
config = SkinConfig()
config.display()

# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)
model.keras_model.metrics_tensors = []

# load weights (originally from MSCOCO model, but here retrained several times)
model.load_weights('/content/drive/My Drive/Colab Notebooks/HD_object_recognition/Mask_RCNN/models/best_so_far.h5', by_name=True) #, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

# train weights (output layers or 'heads')
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=EPOCHS, layers='2+')

## model.train(, layers='heads')  # Train head branches
## model.train(, layers='3+')     # Train resnet stage 3 and up
## model.train(, layers='4+')     # Train resnet stage 4 and up
## model.train(, layers='all')    # Train all layers

# 2.0 Select a saved model and evaluate Mean Average Precision

In [ ]:
saved_model_file = '/content/drive/My Drive/Colab Notebooks/HD_object_recognition/Mask_RCNN/models/best_so_far.h5'

from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import Dataset
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image


class SkinDataset(Dataset):

	def load_dataset(self, dataset_dir, is_train=True):
		# define one class
		self.add_class("dataset", 1, "follicular_bump")
		self.add_class("dataset", 2, "pre_erupted")
		# define data locations
		images_dir = dataset_dir + '/validation images/'
		annotations_dir = dataset_dir + '/validation annots/'
		# find all images
		for filename in listdir(images_dir):
			# extract image id
			image_id = filename[:-4]
			# skip bad images
			if image_id in ['']:
				continue
			# skip all images after 35 if we are building the train set
			if is_train and int(image_id) >= 35:
				continue
			# skip all images before 35 if we are building the test/val set
			if not is_train and int(image_id) < 35:
				continue
			img_path = images_dir + filename
			ann_path = annotations_dir + image_id + '.xml'
			# add to dataset
			self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])

	def extract_boxes(self, filename):
		# load and parse the file
		tree = ElementTree.parse(filename)
		# get the root of the document
		root = tree.getroot()
		# extract each bounding box
		boxes = list()
		for box in root.findall('.//object'):
			name = box.find('name').text
			xmin = int(box.find('./bndbox/xmin').text)
			ymin = int(box.find('./bndbox/ymin').text)
			xmax = int(box.find('./bndbox/xmax').text)
			ymax = int(box.find('./bndbox/ymax').text)
			coors = [xmin, ymin, xmax, ymax, name]
			boxes.append(coors)
		# extract image dimensions
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	def load_mask(self, image_id):
		# get details of image
		info = self.image_info[image_id]
		# define box file location
		path = info['annotation']
		# load XML
		boxes, w, h = self.extract_boxes(path)
		# create one array for all masks, each on a different channel
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			if (box[4] == 'pre_erupted'):
				masks[row_s:row_e, col_s:col_e, i] = 2
				class_ids.append(self.class_names.index('pre_erupted'))
			else:
				masks[row_s:row_e, col_s:col_e, i] = 1
				class_ids.append(self.class_names.index('follicular_bump'))
		return masks, asarray(class_ids, dtype='int32')

	# load an image reference
	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']


# define a configuration for the model
class PredictionConfig(Config):
	NAME = "skinX1.cfg"
	NUM_CLASSES = 3         ## include background (class 0)
	STEPS_PER_EPOCH = 36
	VALIDATION_STEPS = 1
	IMAGE_RESIZE_MODE = 'square'
	IMAGE_MIN_DIM = 1472
	IMAGE_MAX_DIM = 1728    ## should be divisible by 64
	IMAGES_PER_GPU = 1
	LEARNING_RATE = 0.001   ## 0.001


# calculate the mAP for a model on a given dataset
def evaluate_model(dataset, model, cfg):
	APs = list()
	for image_id in dataset.image_ids:
		# load image, bounding boxes and masks for the image id
		image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
		# convert pixel values (e.g. center)
		scaled_image = mold_image(image, cfg)
		# convert image into one sample
		sample = expand_dims(scaled_image, 0)
		# make prediction
		yhat = model.detect(sample, verbose=0)
		# extract results for first sample
		r = yhat[0]
		# calculate statistics, including AP
		AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
		# store
		APs.append(AP)
	# calculate the mean AP across all images
	mAP = mean(APs)
	return mAP


# load the train dataset
train_set = SkinDataset()
train_set.load_dataset('skin data', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

# load the test dataset
test_set = SkinDataset()
test_set.load_dataset('skin data', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

# create config
cfg = PredictionConfig()

# define the model and set to inference mode
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)
model.keras_model.metrics_tensors = []

# load model weights
model.load_weights(saved_model_file, by_name=True)

# evaluate model on training dataset
train_mAP = evaluate_model(train_set, model, cfg)
print("Train mAP: %.3f" % train_mAP)

# evaluate model on test dataset
test_mAP = evaluate_model(test_set, model, cfg)
print("Test mAP: %.3f" % test_mAP)
